On prend le problème de prédiction du débit moyen journalier à une semaine, le critère que l'on va minimiser est un mean square error sur toutes les stations. (discuter)

On choisis de faire de la validation croisée pour faire de la sélection de modèle, et on garde une ensemble de test.  
On a des données de 2010 à 2020 (11 années).  

On garde 2020 comme ensemble de test, et on fait 10 ensembles de validation croisée, et on utilise le split time series de sklearn (prend le début et tester sur l'année suivante), ca fait une erreur calculée sur 9 splits.

Pour les évaluations, la première prédiction est celle du 15 Janvier, que l'on prédit à partir des données du 1 au 7 Janvier, on se place le 8 à 00h00. (on laisse une semaine de données pour calculer les features, et on prédit à J+7).

In [1]:
import pandas as pd
import numpy as np

In [2]:
stations_debit = pd.read_csv("../Data/Base/Stations_Debit.csv")
liste_stations = list(stations_debit["Code station"])
mesures = pd.read_csv("../Data/Base/Mesures.csv")
mesures["Date"] = pd.to_datetime(mesures["Date"], format = "%Y/%m/%d %H:%M:%S")

In [3]:
stations_debit

,Code station,Libelle station,Regime influencé,Altitude du zéro de l'échelle,geometry,Ordre,Cours eau
0,O0200020,La Garonne [partielle] à Saint-Gaudens [Valent...,pas ou faiblement,357.00,POINT (0.7068033357391884 43.09775183863328),0,Garonne
1,O2620010,La Garonne à Verdun-sur-Garonne,pas ou faiblement,90.00,POINT (1.242176195808334 43.85463266047086),1,Garonne
2,O6140010,La Garonne à Lamagistère,pas ou faiblement,46.00,POINT (0.8312973485763152 44.12107242091946),2,Garonne
3,O9000010,La Garonne à Tonneins,pas ou faiblement,0.00,POINT (0.2221525107266094 44.41192952559171),3,Garonne
4,K0030020,La Loire à Issarlès [Pont de La Borie],fortement,881.34,POINT (4.048486196061388 44.81973247281849),0,Loire
5,K0100020,La Loire à Goudet,fortement,0.00,POINT (3.921888161368209 44.88904936219829),1,Loire
6,K0260020,La Loire à Chadrac [Pont du Monteil à Chadrac],fortement en étiage,589.50,POINT (3.903694399697744 45.06795622583292),2,Loire
7,K0550010,La Loire à Bas-en-Basset,fortement,442.00,POINT (4.117990116352597 45.29522866665448),3,Loire
8,K0690010,La Loire à Montrond-les-Bains,fortement,339.11,POINT (4.225355804253323 45.64333335504166),4,Loire
9,K0910050,La Loire à Villerest [Pont de Villerest],fortement,267.51,POINT (4.046217438918319 45.99586067270341),5,Loire


In [4]:
stations_meteo = pd.read_csv("../Data/Base/Stations_Meteo.csv")
colonnes_meteo = ["Pression", "Vent_Nord", "Vent_Est", "Vitesse_vent", "Temperature", "Humidite", "Precipitations"]
liste_stations_meteo = list(stations_meteo["ID"].apply(lambda x: str(x).rjust(5, "0")))
stations_meteo

,ID,Nom,Latitude,Longitude,Altitude,geometry
0,7005,ABBEVILLE,50.136000,1.834000,69,POINT (1.834 50.136)
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.0975 50.57)
2,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.456167 49.18)
3,7037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.181667 49.383)
4,7072,REIMS-PRUNAY,49.209667,4.155333,95,POINT (4.155333 49.209667)
5,7110,BREST-GUIPAVAS,48.444167,-4.412000,94,POINT (-4.412 48.444167)
6,7117,PLOUMANAC'H,48.825833,-3.473167,55,POINT (-3.473167 48.825833)
7,7130,RENNES-ST JACQUES,48.068833,-1.734000,36,POINT (-1.734 48.068833)
8,7139,ALENCON,48.445500,0.110167,143,POINT (0.110167 48.4455)
9,7149,ORLY,48.716833,2.384333,89,POINT (2.384333 48.716833)


## Ensemble de test

In [5]:
mesures_test = mesures[mesures["Date"].apply(lambda x: x.year == 2020)]
mesures_test

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
43824,2020-01-01 00:00:00,60.0,197.0,582.0,1110.0,1.47,6.30,19.4,46.2,88.2,...,4.400000,89.666667,0.0,103053.333333,2.036646,1.921477,2.800000,6.766667,85.666667,0.000000
43825,2020-01-01 02:00:00,56.8,193.0,570.0,1110.0,1.46,6.25,19.2,44.9,88.5,...,3.566667,90.333333,0.0,103043.333333,2.195994,1.842658,2.866667,6.133333,86.000000,0.000000
43826,2020-01-01 04:00:00,61.5,185.0,577.0,1100.0,1.46,6.21,19.0,41.1,88.2,...,3.500000,91.000000,0.0,103030.000000,2.604551,2.185478,3.400000,5.400000,88.000000,0.000000
43827,2020-01-01 06:00:00,68.6,186.0,600.0,1080.0,1.45,6.16,18.9,41.2,87.9,...,2.500000,89.000000,0.0,103010.000000,3.286252,2.161402,3.933333,4.600000,88.666667,0.000000
43828,2020-01-01 08:00:00,56.7,193.0,607.0,1060.0,1.44,6.13,18.8,44.2,87.9,...,4.133333,84.000000,0.0,103043.333333,3.146559,1.816667,3.633333,5.300000,86.333333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48211,2020-12-31 14:00:00,42.3,319.0,865.0,2010.0,0.75,2.94,11.7,34.0,27.5,...,12.600000,40.666667,0.0,101170.000000,1.236535,1.166611,1.700000,10.533333,76.333333,-0.033333
48212,2020-12-31 16:00:00,45.7,311.0,1080.0,2000.0,0.75,2.94,11.6,33.7,43.1,...,12.400000,48.000000,0.0,101130.000000,1.125833,0.650000,1.300000,10.200000,79.000000,-0.100000
48213,2020-12-31 18:00:00,48.6,301.0,1020.0,1950.0,0.75,2.96,11.6,31.7,34.8,...,9.800000,55.333333,0.0,101143.333333,1.898358,1.413275,2.366667,8.466667,84.333333,-0.033333
48214,2020-12-31 20:00:00,48.9,292.0,967.0,1910.0,0.75,2.98,11.5,33.1,37.2,...,7.566667,63.666667,0.0,101130.000000,2.133629,2.012975,2.933333,7.666667,86.666667,0.200000


In [6]:
mesures_moyennes = pd.DataFrame()
for curr_month in range(1, 13):
    mesures_mois = mesures_test[mesures_test["Date"].apply(lambda x: x.month == curr_month)]
    mesures_mois = mesures_mois.groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
    mesures_mois["Jour"] = mesures_mois.index 
    mesures_mois["Date"] = pd.to_datetime("2020/" + str(curr_month).rjust(2, "0") + "/" + mesures_mois["Jour"].apply(lambda x: str(x).rjust(2, "0")))
    mesures_moyennes = pd.concat([mesures_moyennes, mesures_mois])
colonnes = ["Date"] + liste_stations
for code in liste_stations_meteo:
    colonnes += [tpe + "_" + code for tpe in colonnes_meteo]
mesures_test = mesures_moyennes[colonnes]
mesures_test

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
Date,,,,,,,,,,,,,,,,,,,,,
1,2020-01-01,57.383333,191.166667,591.333333,1068.333333,1.440833,6.097500,18.541667,45.708333,87.825000,...,7.333333,78.000000,0.0,103048.611111,1.203607,1.449911,2.841667,8.369444,81.027778,0.000000
2,2020-01-02,55.450000,172.083333,516.250000,839.333333,1.362500,5.724167,17.175000,44.608333,86.675000,...,7.050000,84.916667,0.0,103137.222222,1.300123,1.462018,2.883333,8.275000,81.222222,0.000000
3,2020-01-03,51.900000,168.416667,491.750000,841.666667,1.305000,5.469167,16.350000,44.308333,75.691667,...,9.944444,71.527778,0.0,102926.666667,1.446704,1.428100,2.455556,9.413889,82.472222,0.000000
4,2020-01-04,48.725000,164.750000,461.750000,726.500000,1.272500,5.360000,16.075000,40.075000,61.933333,...,12.658333,61.055556,0.0,102383.611111,0.711918,0.779637,2.236111,11.602778,78.083333,0.000000
5,2020-01-05,47.183333,148.750000,442.833333,681.500000,1.220833,4.981667,15.000000,36.591667,67.475000,...,11.411111,48.972222,0.0,102513.611111,1.023829,0.904314,2.716667,9.925000,82.916667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,4.147222,57.777778,0.0,100928.888889,1.275906,1.048643,2.794444,5.863889,70.055556,-0.011111
28,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,6.627778,61.055556,0.0,99306.666667,-2.840323,-4.304990,7.416667,11.469444,77.194444,0.961111
29,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,8.394444,54.944444,0.0,99947.222222,1.284980,1.626264,2.783333,8.572222,88.666667,0.366667


In [7]:
mesures_test_X = mesures_test
mesures_test_X.to_csv("../Data/Base_journalier/Mesures_Test_X.csv",
                     index = False)

In [8]:
mesures_test_Y = mesures_test.copy()
mesures_test_Y = mesures_test_Y[(mesures_test_Y["Date"].apply(lambda x: x.month != 1)) \
                                | (mesures_test_Y["Date"].apply(lambda x: x.day >= 15))]
mesures_test_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
Date,,,,,,,,,,,,,,,,,,,,,
15,2020-01-15,32.558333,125.416667,303.500000,518.833333,0.962500,3.465000,9.922500,30.966667,33.968333,...,6.547222,78.305556,0.000000,102770.277778,0.924103,0.917797,3.241667,8.405556,75.222222,0.000000
16,2020-01-16,34.158333,116.583333,322.416667,507.250000,0.943333,3.218333,9.325000,29.525000,34.380000,...,7.641667,88.527778,0.000000,102919.722222,1.943863,1.471009,2.447222,9.886111,76.916667,-0.011111
17,2020-01-17,36.283333,128.500000,319.000000,546.166667,0.958333,3.280000,9.489167,27.950000,35.800000,...,9.277778,78.000000,0.027778,102659.166667,-0.028125,1.052540,2.619444,11.672222,81.361111,0.061111
18,2020-01-18,38.525000,138.750000,345.583333,534.583333,0.927500,3.451667,11.091667,28.375000,70.450000,...,9.013889,62.305556,0.000000,102313.055556,0.880842,-2.523291,3.980556,11.269444,69.055556,0.041667
19,2020-01-19,34.666667,138.333333,335.083333,575.500000,0.888333,3.070000,9.660000,26.816667,68.816667,...,8.591667,47.361111,0.000000,102137.777778,1.082962,1.578480,2.894444,7.238889,70.638889,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2020-12-27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,...,4.147222,57.777778,0.000000,100928.888889,1.275906,1.048643,2.794444,5.863889,70.055556,-0.011111
28,2020-12-28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,...,6.627778,61.055556,0.000000,99306.666667,-2.840323,-4.304990,7.416667,11.469444,77.194444,0.961111
29,2020-12-29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,...,8.394444,54.944444,0.000000,99947.222222,1.284980,1.626264,2.783333,8.572222,88.666667,0.366667


In [9]:
mesures_test_Y.to_csv("../Data/Base_journalier/Mesures_Test_Y.csv",
                     index = False)

## Ensemble d'entrainement

In [10]:
mesures_train = mesures[mesures["Date"].apply(lambda x: x.year < 2020)]
mesures_train

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,10.500000,76.000000,0.200000,99500.000000,-7.235633,-2.633555,7.700000,14.200000,90.000000,0.000000
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,10.333333,71.000000,0.133333,99393.333333,-7.506514,-3.769913,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,10.000000,61.000000,0.000000,99180.000000,-7.507236,-6.299319,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,10.533333,50.333333,0.000000,99193.333333,-3.333333,-5.773503,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,11.200000,43.000000,0.000000,99253.333333,-2.600349,-8.685772,9.066667,11.800000,74.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2019-12-31 14:00:00,57.8,205.0,621.0,1110.0,1.53,6.48,19.90,46.2,89.60,...,13.133333,68.666667,0.000000,103073.333333,-0.807839,-1.608539,1.800000,12.366667,70.333333,0.000000
43820,2019-12-31 16:00:00,64.5,206.0,634.0,1110.0,1.53,6.46,19.90,43.7,89.30,...,12.400000,74.000000,0.000000,103040.000000,0.771345,-0.919253,1.200000,12.500000,67.000000,0.000000
43821,2019-12-31 18:00:00,67.2,201.0,635.0,1100.0,1.52,6.45,19.70,42.8,89.00,...,8.800000,84.666667,0.000000,103066.666667,-1.260781,1.189486,1.733333,10.833333,74.333333,0.000000
43822,2019-12-31 20:00:00,56.3,198.0,625.0,1100.0,1.50,6.42,19.50,45.2,88.70,...,6.666667,89.666667,0.000000,103073.333333,1.542690,1.838507,2.400000,9.100000,81.000000,0.000000


In [11]:
mesures_moyennes = pd.DataFrame()
liste_annees = np.unique(mesures_train["Date"].apply(lambda x: x.year))
for curr_annee in liste_annees:
    mesures_annee = mesures_train[mesures_train["Date"].apply(lambda x: x.year == curr_annee)]
    for curr_month in range(1, 13):
        mesures_mois = mesures_annee[mesures_annee["Date"].apply(lambda x: x.month == curr_month)]
        mesures_mois = mesures_mois.groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
        mesures_mois["Jour"] = mesures_mois.index 
        mesures_mois["Date"] = pd.to_datetime(str(curr_annee) + "/" + str(curr_month).rjust(2, "0") + "/" + mesures_mois["Jour"].apply(lambda x: str(x).rjust(2, "0")))
        mesures_moyennes = pd.concat([mesures_moyennes, mesures_mois])
colonnes = ["Date"] + liste_stations
for code in liste_stations_meteo:
    colonnes += [tpe + "_" + code for tpe in colonnes_meteo]
mesures_train = mesures_moyennes[colonnes]
mesures_train

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
Date,,,,,,,,,,,,,,,,,,,,,
1,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.838889,54.916667,0.027778,99468.888889,-3.102706,-10.306272,11.608333,12.455556,78.805556,0.527778
2,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.402778,62.138889,0.000000,100958.611111,0.599922,-5.787958,6.166667,11.091667,57.361111,0.005556
3,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.850000,66.916667,0.000000,101789.722222,0.935041,1.516902,3.294444,6.633333,63.944444,0.000000
4,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.375000,83.027778,0.022222,101092.500000,1.822732,1.671234,2.491667,8.925000,85.000000,0.066667
5,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.783333,87.361111,0.333333,99866.666667,1.063713,1.520201,2.219444,10.800000,93.111111,1.127778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.713889,58.944444,0.000000,102100.555556,1.235221,1.197670,2.586111,9.486111,83.777778,0.027778
28,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.244444,72.500000,0.000000,102638.611111,1.344905,1.270057,2.541667,9.491667,81.222222,0.000000
29,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.761111,82.638889,0.000000,103028.333333,1.199456,1.430483,2.675000,9.586111,73.861111,0.000000


In [12]:
mesures_train_X = mesures_train.copy()
mesures_train_X.to_csv("../Data/Base_journalier/Mesures_Train_X.csv",
                     index = False)

In [13]:
mesures_train_Y = mesures_train.copy()
mesures_train_X.to_csv("../Data/Base_journalier/Mesures_Train_Y.csv",
                     index = False)

In [14]:
mesures_train_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
Date,,,,,,,,,,,,,,,,,,,,,
1,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.838889,54.916667,0.027778,99468.888889,-3.102706,-10.306272,11.608333,12.455556,78.805556,0.527778
2,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.402778,62.138889,0.000000,100958.611111,0.599922,-5.787958,6.166667,11.091667,57.361111,0.005556
3,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.850000,66.916667,0.000000,101789.722222,0.935041,1.516902,3.294444,6.633333,63.944444,0.000000
4,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.375000,83.027778,0.022222,101092.500000,1.822732,1.671234,2.491667,8.925000,85.000000,0.066667
5,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.783333,87.361111,0.333333,99866.666667,1.063713,1.520201,2.219444,10.800000,93.111111,1.127778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.713889,58.944444,0.000000,102100.555556,1.235221,1.197670,2.586111,9.486111,83.777778,0.027778
28,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.244444,72.500000,0.000000,102638.611111,1.344905,1.270057,2.541667,9.491667,81.222222,0.000000
29,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.761111,82.638889,0.000000,103028.333333,1.199456,1.430483,2.675000,9.586111,73.861111,0.000000


## Ensembles de validation croisée

In [15]:
mesures_train_X = pd.read_csv("../Data/Base_journalier/Mesures_Train_X.csv")
mesures_train_X["Date"] = pd.to_datetime(mesures_train_X["Date"], format = "%Y/%m/%d")
mesures_train_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.838889,54.916667,0.027778,99468.888889,-3.102706,-10.306272,11.608333,12.455556,78.805556,0.527778
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.402778,62.138889,0.000000,100958.611111,0.599922,-5.787958,6.166667,11.091667,57.361111,0.005556
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.850000,66.916667,0.000000,101789.722222,0.935041,1.516902,3.294444,6.633333,63.944444,0.000000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.375000,83.027778,0.022222,101092.500000,1.822732,1.671234,2.491667,8.925000,85.000000,0.066667
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.783333,87.361111,0.333333,99866.666667,1.063713,1.520201,2.219444,10.800000,93.111111,1.127778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.713889,58.944444,0.000000,102100.555556,1.235221,1.197670,2.586111,9.486111,83.777778,0.027778
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.244444,72.500000,0.000000,102638.611111,1.344905,1.270057,2.541667,9.491667,81.222222,0.000000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.761111,82.638889,0.000000,103028.333333,1.199456,1.430483,2.675000,9.586111,73.861111,0.000000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.797222,85.611111,0.000000,103024.444444,1.321962,1.520627,2.986111,6.375000,73.944444,0.000000


In [16]:
train_split_X = pd.DataFrame()
for i in range(9):
    train_split_X["Train_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year <= 2010+i)
for i in range(9):
    train_split_X["Test_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year == 2010+i+1)
train_split_X

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
4,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3648,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3649,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3650,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [17]:
train_split_X.to_csv("../Data/Base_journalier/Index_CV_X.csv",
                    index=False)

In [18]:
mesures_train_Y = pd.read_csv("../Data/Base_journalier/Mesures_Train_Y.csv")
mesures_train_Y["Date"] = pd.to_datetime(mesures_train_Y["Date"], format = "%Y/%m/%d")
mesures_train_Y

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Vent_Nord_07761,Vent_Est_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,...,10.838889,54.916667,0.027778,99468.888889,-3.102706,-10.306272,11.608333,12.455556,78.805556,0.527778
1,2010-01-02,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,...,5.402778,62.138889,0.000000,100958.611111,0.599922,-5.787958,6.166667,11.091667,57.361111,0.005556
2,2010-01-03,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,...,4.850000,66.916667,0.000000,101789.722222,0.935041,1.516902,3.294444,6.633333,63.944444,0.000000
3,2010-01-04,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,...,5.375000,83.027778,0.022222,101092.500000,1.822732,1.671234,2.491667,8.925000,85.000000,0.066667
4,2010-01-05,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,...,6.783333,87.361111,0.333333,99866.666667,1.063713,1.520201,2.219444,10.800000,93.111111,1.127778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,...,12.713889,58.944444,0.000000,102100.555556,1.235221,1.197670,2.586111,9.486111,83.777778,0.027778
3648,2019-12-28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,...,11.244444,72.500000,0.000000,102638.611111,1.344905,1.270057,2.541667,9.491667,81.222222,0.000000
3649,2019-12-29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,...,7.761111,82.638889,0.000000,103028.333333,1.199456,1.430483,2.675000,9.586111,73.861111,0.000000
3650,2019-12-30,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,...,6.797222,85.611111,0.000000,103024.444444,1.321962,1.520627,2.986111,6.375000,73.944444,0.000000


In [19]:
train_split_Y = pd.DataFrame()
for i in range(9):
    train_split_Y["Train_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year <= 2010+i)
    train_split_Y["Train_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False # On enlève les 14 premiers jours
    
for i in range(9):
    train_split_Y["Test_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year == 2010+i+1)
    train_split_Y["Test_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False  # On enlève les 14 premiers jours
train_split_Y

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3648,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3649,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3650,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [20]:
train_split_Y.to_csv("../Data/Base_journalier/Index_CV_Y.csv",
                    index=False)